# Sprint 機械学習フロー

# 【問題1】クロスバリデーション
事前学習期間では検証用データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。

事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
from scipy import stats

In [2]:
# trainデータセットの準備
df = pd.read_csv("application_train.csv")

In [3]:
# 欠損値の対応
# オブジェクトは最頻値で埋める
# 数字は平均値で埋める

# for + fillnaでオブジェクト欠損値を最頻値で埋める
# modeはの数値にアクセスするには[0]スライスが必要
df_obj = df.select_dtypes(include = ['object'])
for col_obj in df_obj.columns:
    df[col_obj] = df[col_obj].fillna(df_obj[col_obj].mode()[0])
    
# 数値の列名を取得
df_num = df.select_dtypes(include = ['float64', 'int64'])
for col_num in df_num.columns:
    df[col_num] = df[col_num].fillna(df_num[col_num].mean())

In [4]:
# 各オブジェクトをダミー処理
df_dummy = list(df.select_dtypes(include='object').columns)
df = pd.concat([df, pd.get_dummies(df[df_dummy])], axis = 1, sort = False)
df.drop(columns = df[df_dummy], inplace = True)

In [5]:
df.shape

(307511, 246)

In [6]:
# テストデータの準備
df_test = pd.read_csv("application_test.csv")

In [7]:
# テストデータ
# 欠損値の前処理（オブジェクトは最頻値、数字は平均で埋める）
# オブジェクトは最頻値
df_test_obj = df_test.select_dtypes(include = ['object'])
for col_test_obj in df_test_obj.columns:
    df_test[col_test_obj] = df_test[col_test_obj].fillna(df_test_obj[col_test_obj].mode()[0])
# 数値は平均値
df_test_num = df_test.select_dtypes(include = ['float64', 'int64'])
for col_test_num in df_test_num.columns:
    df_test[col_test_num] = df_test[col_test_num].fillna(df_test_num[col_test_num].mean())
# 欠損値の有無を再確認
df_test.isnull().sum().sum()

0

In [8]:
# テストデータ
# オブジェクトのone_hot化
df_test_dummy = list(df_test.select_dtypes(include='object').columns)
df_test = pd.concat([df_test, pd.get_dummies(df_test[df_test_dummy])], axis = 1, sort = False)
df_test.drop(columns = df_test[df_test_dummy], inplace = True)

In [9]:
# トレインとテストデータのshapeを確認
print(df.shape)
print(df_test.shape)

(307511, 246)
(48744, 242)


In [10]:
# trainとtestで列数が異なるため、互いに含まれない列を増やす
# 本来含まれていない列の値は０に設定

#testの列を増加
for column_df in df.columns:
    if column_df not in df_test.columns:
        df_test[column_df] = 0
# trainの列を増加
for column_test in df_test.columns:
    if column_test not in df.columns:
        df[column_test] = 0
#testデータに追加されたTARGETは削除
df_test.drop(columns="TARGET", inplace=True)

print(df.shape)
print(df_test.shape)

(307511, 246)
(48744, 245)


In [11]:
# Xとyに分割
y = df[["TARGET"]].copy()
df.drop(columns = "TARGET", inplace = True)
X = df.copy()

In [19]:
# 各特徴量の標準化
from sklearn.preprocessing import StandardScaler

# ndarray変換
X_array = X.values
y_array = y.values
X_test = df_test.values

# 標準化
scaler_X = StandardScaler()
X_train_std = scaler_X.fit_transform(X_array)
X_test_std = scaler_X.transform(X_test)

# yの一次元化(ロジスティック回帰で必要)
y_train = np.ravel(y_array)

In [23]:
# kfold + cross validation、モデルはrandomforest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
rfc = RandomForestClassifier()
kf = KFold(n_splits=5)
result = cross_val_score(rfc, X_train_std, y_train, cv = kf, scoring = "roc_auc")

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-package

In [34]:
print("cross_val_scoreを使用したvalidの結果：\n", result)

cross_val_scoreを使用したvalidの結果：
 [0.6127655  0.63067932 0.62084629 0.63430622 0.62124432]


In [30]:
# cross_val_scoreを使用せず
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
score_train = []
score_valid = []
for train_index, valid_index in kf.split(X_train_std):
    # 分割
    X_train_std_kf, X_std_valid_kf = X_train_std[train_index], X_train_std[valid_index]
    y_train_kf, y_valid_kf = y_train[train_index], y_train[valid_index]
    # randomforest
    rfc_kf = RandomForestClassifier()
    # result = cross_val_score(rfc, X_train_std, y_train, cv = kf, scoring = "roc_auc")
    rfc_kf.fit(X_train_std_kf, y_train_kf)
    y_train_predict_rfc_kf = rfc_kf.predict_proba(X_train_std_kf)
    # ROCによる評価
    # trainの結果
    print("train : roc_auc_score : ", roc_auc_score(y_train_kf, y_train_predict_rfc_kf[:, 1]))
    score_train.append(roc_auc_score(y_train_kf, y_train_predict_rfc_kf[:, 1]))
    # validの結果
    y_valid_predict_rfc_kf = rfc_kf.predict_proba(X_std_valid_kf)
    print("valid : roc_auc_score : ", roc_auc_score(y_valid_kf, y_valid_predict_rfc_kf[:, 1]))
    score_valid.append(roc_auc_score(y_valid_kf, y_valid_predict_rfc_kf[:, 1]))

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


train : roc_auc_score :  0.9997955038455697
valid : roc_auc_score :  0.6169337855306426


/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


train : roc_auc_score :  0.9997755394966221
valid : roc_auc_score :  0.6341283909102585


/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


train : roc_auc_score :  0.9997326446616897
valid : roc_auc_score :  0.6280842078460791


/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


train : roc_auc_score :  0.9998000756910054
valid : roc_auc_score :  0.632757090081961


/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


train : roc_auc_score :  0.9997365110456068
valid : roc_auc_score :  0.6228362252374051


 # 【問題2】グリッドサーチ
これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。最適なパラメータを探していくことを パラメータチューニング と呼びます。パラメータチューニングをある程度自動化する単純な方法としては グリッドサーチ があります。

scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。

sklearn.model_selection.GridSearchCV — scikit-learn 0.21.3 documentation

GridSearchCVクラスには引数としてモデル、探索範囲、さらにクロスバリデーションを何分割で行うかを与えます。クロスバリデーションの機能も含まれているため、これを使用する場合はKFoldクラスを利用する必要はありません。

In [35]:
from sklearn.model_selection import GridSearchCV
param = {"min_samples_leaf":[5, 10], "max_depth":[1, 5,10]}
gs_rfc = GridSearchCV(RandomForestClassifier(),
                  param_grid=param, cv=5, scoring='roc_auc')
gs_rfc.fit(X_train_std, y_train)
y_train_predict_gs_rfc = gs_rfc.predict_proba(X_train_std)
# ROCによる評価
# validの結果
print("train : roc_auc_score : ", roc_auc_score(y_train, y_train_predict_gs_rfc[:, 1]))
score_train.append(roc_auc_score(y_train, y_train_predict_gs_rfc[:, 1]))

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-package

train : roc_auc_score :  0.7670437235456998


# 【問題3】Kernelからの調査
KaggleのKernelから様々なアイデアを見つけ出して、列挙してください。

* Nested Cross Validationを使用する  
→上記はNon-nested Cross Validation、内部調整と外部検証に別ける方が汎用性高い


* KFoldを指定する際にshuffle=Trueにする  
→Trueにしないでrandom_stateを指定すると、random_stateが変化しても抜き出すインデックスが変化しない


* StratifiedKFoldを使用する
→通常のKFoldだとtarget（０、１）の割合が均等でない


 * グリッドサーチ する際、最初に１０の累乗など大まかに調べる  
 →細かく指定すると計算コストが高いため、最初は粗く。

 # 【問題4】高い汎化性能のモデル作成
問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。

その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

In [53]:
# nested cross validation
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
param_nes = {"min_samples_leaf":[5, 10], "max_depth":[1, 5,10]}
gs_rfc_nes = GridSearchCV(RandomForestClassifier(),
                          param_grid=param_nes, cv=skf, scoring='roc_auc')
result_nes = cross_validate(gs_rfc_nes, X_train_std, y_train, cv=skf,
                            scoring = "roc_auc", return_train_score=True,
                            return_estimator=True, error_score=True)

pd.DataFrame(result_nes)

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-package

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-package

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-package

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-package

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-package

,fit_time,score_time,estimator,test_score,train_score
0,112.915040,0.207939,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.710898,0.773870
1,114.598488,0.143234,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.719950,0.774218
2,113.683296,0.139010,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.708646,0.776433
3,111.973838,0.132667,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.720735,0.770625
4,107.954829,0.129212,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.724601,0.772530


* Nested Cross Validationによりスコアは低下した  
→学習ようのサンプル数が減るためスコアは低下する  
　一方で汎用性は上がる（らしい）

# 【問題5】最終的なモデルの選定
最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。

In [112]:
# nested cross validationのベストパラメータを抽出
predictor_idx = pd.DataFrame(result_nes)["test_score"].idxmax()
best_estimator = result_nes["estimator"][predictor_idx].best_estimator_
best_estimator

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [113]:
# 予測
y_test_predict_proba = best_estimator.predict_proba(X_test_std)
y_test_predict_proba

array([[0.84445293, 0.15554707],
       [0.89960907, 0.10039093],
       [0.93625226, 0.06374774],
       ...,
       [0.93322672, 0.06677328],
       [0.94967151, 0.05032849],
       [0.88748915, 0.11251085]])

In [115]:
# SK_ID_CURRとtest_predictをdataframe化
# TARGET列名を指定通りに修正
test_submit_df = pd.concat([df_test["SK_ID_CURR"],
                            pd.DataFrame(y_test_predict_proba[:, 1], columns=['TARGET'])], axis=1)

# kaggleに提出するcsv作成
# インデックスをIDに変更
test_submit_df.to_csv("submit_200204.csv", index=False)

In [116]:
# 提出ファイルを出力し中身確認
pd.read_csv("submit_200204.csv").head()

,SK_ID_CURR,TARGET
0,100001,0.155547
1,100005,0.100391
2,100013,0.063748
3,100028,0.071326
4,100038,0.197640
